## Import and install

In [97]:
#!conda install -c conda-forge beautifulsoup4 --yes 
from bs4 import BeautifulSoup

In [98]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

# Question 1
## Download data

In [99]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results = requests.get(url)

soup = BeautifulSoup(results.text, "html.parser")

## Process data

In [116]:
soup.table.tbody

# Create Dataframe

row = []
for tr in soup.table.tbody.find_all('tr'):
    td = tr.find_all('td')
    if len(td)==0:
        th = tr.find_all('th')
        col=[str(th[0].string).rstrip(),str(th[1].string).rstrip(),str(th[2].string).rstrip()]
    else:
        row.append([str(td[0].string).rstrip(),str(td[1].string).rstrip(),str(td[2].string).rstrip()]) 

#col = ['Postcode','Borough','Neighbourhood']
df_pc = pd.DataFrame(row,columns=col)

# Remove NA

df_pc = df_pc[df_pc[col[1]] != 'Not assigned']



# Group Postcode

def ClearNeighbourhood(x):
    nebstr = x.Neighbourhood
    neblist = nebstr.split(',')
    while 'Not assigned' in neblist:
        neblist.remove('Not assigned')
    while 'None' in neblist:
        neblist.remove('None') 
    while '' in neblist:
        neblist.remove('')        
    nebstr = ','.join(str(e) for e in neblist)
    return nebstr

df_pc = df_pc.groupby([col[0],col[1]])[col[2]].agg(lambda x: ','.join(x)).reset_index(name ='col[2]')
df_pc.columns = col


df_pc[col[2]] = df_pc.apply(ClearNeighbourhood, axis = 1)
# asign Neighbourhood by Borough
df_pc[col[2]] = np.where(df_pc[col[2]] == '', df_pc[col[1]] , df_pc[col[2]] )

df_pc




,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,Scarborough
1,M1C,Scarborough,Scarborough
2,M1E,Scarborough,Guildwood
3,M1G,Scarborough,Scarborough
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough
6,M1K,Scarborough,East Birchmount Park
7,M1L,Scarborough,Scarborough
8,M1M,Scarborough,Scarborough Village West
9,M1N,Scarborough,Cliffside West


In [117]:
df_pc.shape

(103, 3)

# Question 2
## Get Georaphical from url 

In [141]:
# Get Geographical coordinates 

df_gc = pd.read_csv("https://cocl.us/Geospatial_data")
df_gc.columns = ['Postcode','Latitude','Longitude']
df = pd.merge(df_pc,df_gc, how='inner', on = 'Postcode')

#df = pd.merge(df_gc, df_pc, how='inner', left_on = 'Postcode',right_on = 'Post code')
df.head()


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,Scarborough,43.806686,-79.194353
1,M1C,Scarborough,Scarborough,43.784535,-79.160497
2,M1E,Scarborough,Guildwood,43.763573,-79.188711
3,M1G,Scarborough,Scarborough,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## geocoder longtime to respond (disabled)


In [ ]:
#!conda install -c conda-forge geocoder --yes 
import geocoder # import geocoder

df_pc['Latitude'],df_pc['Longitude'] =  df_pc.apply(GetLocation, axis = 1) 

# initialize your variable to None
def GetLocation(x):
    lat_lng_coords = None
    postal_code = x['Postcode']
    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude


# Question 3

In [ ]:
df_toronto = df[df['Borough'].astype(str).str.find('Toronto')!=-1]
df_toronto